In [1]:
import Tkinter, tkFileDialog, os
from Libs import LVQLib
from Libs import ExpWriter as ew

In [30]:
class LVQUI(Tkinter.Tk):    
    """
    
    Views:
        1) Load in samples
        2) Label Classifications -- show the network
        3) Training & save screen

    """
    def __init__(self, parent):
        self.viewdex = 0        # current view index
        self.curView = []
        self.parent  = parent
        self.state   = { 'savePath': None, 'vectors': [] }
        self.views   = [ 'Load Vectors', 'Label Neurons', 'Train & Save' ]
        self.inits   = [ self.loadViewInit, self.labelViewInit, self.trainViewInit ]
        
        Tkinter.Tk.__init__(self, parent)
        self.initialize()
        
    def initialize(self):
        self.grid()
        self.grid_columnconfigure(0, weight=2)
        self.resizable(False, True)
        
        self.screen_title = Tkinter.Label(self, text=self.views[self.viewdex], anchor='center')
        self.listBox      = Tkinter.Listbox(self, width=75)
        self.forwardBtn   = Tkinter.Button(self, text=" Next  >>>", command=self.nextView,
                                           width=10)
        self.backwardBtn  = Tkinter.Button(self, text="<<<  Back ", command=self.prevView,
                                           width=10)
        # Grid Positioning
        self.screen_title.grid(column=4, row=1, columnspan=4, sticky='W', padx=20)
        self.forwardBtn.grid(column=16,  row=1, columnspan=4, sticky='E', padx=40)
        self.backwardBtn.grid(column=0,  row=1, columnspan=4, sticky='W', padx=40)
        self.listBox.grid(column=4,      row=3, columnspan=6, sticky='W', padx=20, pady=10)
        
        self.loadViewInit()
        self.update()
        
        self.geometry(self.geometry()) # comment out for auto resizing.
        
    def fileWrapper(self):
        self.getFileDialogue()
        self.listBox.update_idletasks()
        
    def getFileDialogue(self):
        """
        Open a file chooser and extract the path 
        (add to filepaths for input vectors).
        """
        filey = tkFileDialog.askopenfile(parent=self, mode='r', 
                                         title="Audio Sample")
        if (filey != None):
            self.addFile(filey.name)
            self.updateList(self.listBox)
            self.clearView()
            self.loadViewInit()
            filey.close()
            
        self.update()
        
    def nextView(self):
        if (self.viewdex < len(self.views) - 1):
            self.clearView()
            self.viewdex += 1
            self.screen_title['text'] = self.views[self.viewdex]
            
        self.update()
        
    def prevView(self):
        if (self.viewdex > 0):
            self.clearView()
            self.viewdex -= 1
            self.screen_title['text'] = self.views[self.viewdex]

        self.update()
    
    def clearView(self):
        """
        Clear the canvas from all non-permanent elements
        """
        for i in self.curView:
            i.grid_forget()
            
        self.update()
        self.curView = []
        
    # Temporary
    def updater(self):
        self.clearView()
        self.update()
    
    def loadViewInit(self):
        """
        Load the view (1): 
            Load samples for training
            (and vector initialization of the neurons)
        """
        self.fileLabel = Tkinter.Label(self,  text="Files Imported:")
        self.fileBtn   = Tkinter.Button(self, text="Add Vector", command=self.fileWrapper)
        self.removeBtn = Tkinter.Button(self, text="Update",     command=self.updater)
        
        self.curView = [ self.fileLabel, self.fileBtn, self.removeBtn ]
        
        self.curView[0].grid(column=0, row=2, columnspan=1, sticky='N', pady=2)
        self.curView[1].grid(column=0, row=3, columnspan=1, sticky='N', pady=2)
        self.curView[2].grid(column=0, row=4, columnspan=1, sticky='N', pady=2)
        
        self.update()
        
    def labelViewInit(self):
        """
        Load the view (2): 
            Label neuron classifiers
        """
        print "LABEL view"
    
    def trainViewInit(self):
        """
        Load the view (3):
            Training Parameters & Output / Progress
        """
        print "TRAIN view"
    
    def updateList(self, listbox):
        files = self.state['vectors']
        listbox.insert('end', files[-1])
        listbox.update_idletasks()
    
    def addFile(self, filepath):
        self.state['vectors'].append(filepath)
        
    def removeFile(self, filepath):
        self.state['vectors'].remove(filepath)
        
    def setSavePath(self, filepath):
        self.state['savePath'] = filepath

In [31]:
if __name__ == '__main__':
    gui = LVQUI(None)
    gui.title('LVQ Drum Training')
    gui.mainloop()